In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import statistics
import sklearn
import sklearn.metrics as sm

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier

from sklearn.model_selection import KFold, cross_val_score
pd.options.display.max_columns = 130

In [ ]:
dataset = pd.read_csv("../input/tabular-playground-series-oct-2021/train.csv")
testset = pd.read_csv("../input/tabular-playground-series-oct-2021/test.csv")

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
target = dataset['target']
dataset = dataset.drop(columns=['id','target'])

In [ ]:
dataset.isnull().sum().sort_values(ascending=False)

In [ ]:
dataset['std'] = dataset.std(axis=1)
dataset['var'] = dataset.var(axis=1)
dataset['min'] = dataset.min(axis=1)
dataset['max'] = dataset.max(axis=1)

dataset.head()

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(dataset,target, test_size = 0.3, random_state = 60,shuffle=True)
print(len(X_train))
print(len(X_test))

In [ ]:
params = {
    'max_depth': 6,
    'n_estimators': 5500,
    'subsample': 0.6000000000000001,
    'colsample_bytree': 0.2,
    'colsample_bylevel': 0.4,
    'min_child_weight': 0.0475667709098205,
    'reg_lambda': 50.33144833870577,
    'reg_alpha': 0.01634917276171278,
    'gamma': 5.507875585868313,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'use_label_encoder': False
}

In [ ]:
xgb = XGBClassifier(**params)
xgb.fit(X_train, Y_train)

test_predict = xgb.predict(X_test)


In [ ]:
k_fold = KFold(n_splits=10, shuffle=True, random_state=0)
k_fold_results = cross_val_score(xgb, X_train,  Y_train, cv=k_fold, n_jobs=1)
print(k_fold_results)
print("Avg accuracy: {}".format(k_fold_results.mean()))

test_predict = xgb.predict(X_test)

In [ ]:
y_pred_proba = xgb.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(Y_test,  y_pred_proba)
auc = metrics.roc_auc_score(Y_test, y_pred_proba)
print(auc)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
testset.isnull().sum().sort_values(ascending = False)


In [ ]:
test_id = testset['id']
testset= testset.drop(columns=['id'])


In [ ]:
testset['std'] = testset.std(axis=1)
testset['var'] = testset.var(axis=1)
testset['min'] = testset.min(axis=1)
testset['max'] = testset.max(axis=1)

In [ ]:
y_prdict =xgb.predict_proba(testset)[:, 1]

In [ ]:
submission = pd.DataFrame(list(zip(test_id, y_prdict)),
               columns =['id', 'target'])
submission.head(20)

In [ ]:
submission.to_csv('submission.csv', index=False)